In [15]:
#import plotly.express as px
#import plotly as plotly
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [16]:
from styleframe import StyleFrame as sf

In [80]:
df = pd.read_excel(r'totales_complejos_1sem_2021.xlsx')
#df = df.loc[['ncomplejo'], ['nsubcomplejo'], ['ngrupo'], ['partim']]
df = df[['ncomplejo', 'nsubcomplejo','ngrupo','complejo_exportador','partim','fob_aactm']]

In [91]:
i_son_nan = []
for i, a in enumerate(df.nsubcomplejo):
    if (np.isnan(a)):
        i_son_nan.append(i)
df_complejos = df.iloc[i_son_nan]

i_son_nan = []
for i, a in enumerate(df.ngrupo):
    if (np.isnan(a)):
        i_son_nan.append(i)
df_subcomplejos = df.iloc[i_son_nan]

for complejo in df_complejos:
    for subc in df_subcomplejos:
        if(subc == complejo):
            del subc

df_subcomplejos


,ncomplejo,nsubcomplejo,ngrupo,complejo_exportador,partim,fob_aactm
0,0,NaN,NaN,Total exportaciones:,100.0,35373
1,1,NaN,NaN,---Sector oleaginoso,37.0,13100
2,1,1.0,NaN,------Complejo soja,33.6,11900
8,1,3.0,NaN,------Complejo girasol,1.9,689
12,1,2.0,NaN,------Complejo maní,1.3,452
...,...,...,...,...,...,...
158,17,NaN,NaN,---Complejo yerba mate,0.1,38
159,17,1.0,NaN,------Complejo yerba mate,0.1,38
162,19,NaN,NaN,---Complejo equino,0.1,37
163,19,1.0,NaN,------Complejo equino,0.1,37


In [189]:
def busca_padres (df, ncomplejo, nsubcomplejo):
    ncomplejo = ncomplejo.values[0]
    nsubcomplejo = nsubcomplejo.values[0]

    if ncomplejo == 0:
        return ''
    elif np.isnan(nsubcomplejo):
        padre = df.loc[(df.ncomplejo == ncomplejo)&(np.isnan(df.nsubcomplejo))]
        padre = padre.complejo_exportador.values[0]
        
        return padre
    else:
        padre = df.loc[(df.ncomplejo == ncomplejo)&(df.nsubcomplejo == nsubcomplejo)&(np.isnan(df.ngrupo))]
        padre = padre.complejo_exportador.values[0]
        return padre

In [161]:
def lista_padres (df):
    padres = []
    for i in range(len(df)):
        complejo = df.iloc[[i]]
        padres.append(busca_padres(df, complejo.ncomplejo, complejo.nsubcomplejo))
    return padres
            

In [190]:
lista_padres(df)

['',
 '---Sector oleaginoso',
 '------Complejo soja',
 '------Complejo soja',
 '------Complejo soja',
 '------Complejo soja',
 '------Complejo soja',
 '------Complejo soja',
 '------Complejo girasol',
 '------Complejo girasol',
 '------Complejo girasol',
 '------Complejo girasol',
 '------Complejo maní',
 '------Complejo maní',
 '------Complejo maní',
 '------Complejo maní',
 '------Complejo olivícola',
 '------Complejo olivícola',
 '------Complejo olivícola',
 '------Complejo olivícola',
 '---Sector cerealero',
 '------Complejo maicero',
 '------Complejo maicero',
 '------Complejo maicero',
 '------Complejo triguero',
 '------Complejo triguero',
 '------Complejo triguero',
 '------Complejo cebada',
 '------Complejo cebada',
 '------Complejo cebada',
 '------Complejo cebada',
 '------Complejo arrocero',
 '------Complejo arrocero',
 '------Complejo arrocero',
 '---Complejo automotriz',
 '------Complejo automotriz',
 '------Complejo automotriz',
 '------Complejo automotriz',
 '------Comp

In [17]:
df1 = sf.read_excel('complexp_est_porcentual_2017_2020.xlsx', read_style=True, use_openpyxl_styles=False)
df1.columns = ['Complejos','2017','2018','2019','2020']
df1 = df1.iloc[4:68]
df1 =  df1.reset_index(drop=True)

In [18]:
def listaColumnas(df):
    columnas = []
    for e in list(df.columns):
        columnas.append(str(e))
    return columnas

def indiceSinEspacios(lista, buscar):
    for idx, e in enumerate(lista):
        if(str(e).strip()==buscar):
            return idx

def datosComplejo (df, complejo, anio):
    anio = str(anio)
    complejo = str(complejo)
    indice_anio = listaColumnas(df).index(anio)
    indice_del_complejo = indiceSinEspacios(list(df[str(df.columns[0])]), complejo)
    return float(df.loc[indice_del_complejo][indice_anio])


-si es negrita su padre es pples complejos

-si no es negrita su padre es el ultimo elemento cuyo padre es pples complejos

-si tiene una tabulacion su padre es el ultimo elemento cuyo padre es el ultimo elemento de pples complejos

-Si tiene 2 tab su padre es el ultimo

In [19]:
def datos_para_teemap(df, anio, redondeo):
    datos = []
    nombres = []
    padres = ['','Total de exportaciones']
    ultimo_complejo = ''
    ultimo_padre = 'A'
    ultima_negrita = 'B'
    ultima_1tab = 'C'
    for complejo in df.Complejos:
        ultimo_complejo = str(complejo).strip()
        
        datos.append(round(datosComplejo (df, ultimo_complejo, anio),redondeo))
        nombres.append(ultimo_complejo)
        if(str(complejo)!= 'Resto de exportaciones' and str(complejo) != 'Total de exportaciones' and str(complejo) != 'Principales complejos'):
            if(complejo.style.bold):
                #print('N ' + ultimo_complejo)
                ultimo_padre = 'Principales complejos'
                padres.append(ultimo_padre)
                ultima_negrita = ultimo_complejo
                ultima_1tab = ultimo_complejo
            elif(complejo.style.bold == False):
                if(str(complejo)[3] != ' '):
                    #print('->' + ultimo_complejo)
                    ultimo_padre = ultima_negrita
                    
                    padres.append(ultimo_padre)
                    ultima_1tab = ultimo_complejo
                else:
                    #print('->->' + ultimo_complejo)
                    ultimo_padre = ultima_1tab
                    padres.append(ultimo_padre)
        elif(str(complejo) == 'Resto de exportaciones'):
            padres.append('Total de exportaciones')
    return {
        'nombres': nombres,
        'padres': padres,
        'datos': datos 
    }


        

In [20]:
a = 35
b = 30
print(
    datos_para_teemap(df1,2020,2)['nombres'][b:a],
    datos_para_teemap(df1,2020,2)['padres'][b:a],
    datos_para_teemap(df1,2020,2)['datos'][b:a]
)

['Carne bovina', 'Cueros bovinos', 'Complejo lácteo', 'Complejo petrolero-petroquímico', 'Petróleo y gas'] ['Complejo carne y cuero bovinos', 'Complejo carne y cuero bovinos', 'Sector bovino', 'Principales complejos', 'Complejo petrolero-petroquímico'] [0.25, 0.8, 0.1, 6.67, 5.7]


In [21]:
df = pd.read_excel('complexp_est_porcentual SOLO2020.xls')
#Se queda con la primera fila de los valores
df = df.iloc[[0]]
#Construye una df con los nombres de los complejos/sectores
df = df[[
  'Sector oleaginoso',
  'Sector cerealero',
  'Complejo automotriz',
  'Sector minero metalífero y litio',
  'Sector bovino',
  'Complejo petrolero-petroquímico',
  'Sector frutícola',
  'Complejo pesquero',
  'Sector hortícola',
  'Complejo farmacéutico',
  'Complejo forestal',
  'Complejo avícola',
  'Complejo textil',
  'Complejo tabacalero',
  'Complejo miel',
  'Complejo azucarero',
  'Complejo yerba mate',
  'Complejo té',
  'Complejo equino',
  'Resto de exportaciones'
]]
#Lista vacía
datos = []
#Cada dato individual que esté en la primera fila de df (convdertido en float y redondeado a 2 dec) será agregado en la lista datos
for dato in df.iloc[[0]]:
  datos.append(round(float(df[dato]),2))

In [22]:
def convert(s):
    # initialization of string to ""
    new = ""
    # traverse in the string 
    for x in s:
        new += x 
    # return string 
    return new

In [23]:
#Agarra los nombres de las columnas, las convierte en string con la función del anterior chunk y los convierte en lista para agregarlo en la lista nombres.
nombres = []
for s in list(df.columns):
  nombres.append(convert(s))
etiquetas = ['']+['Exportaciones']+nombres+['Otros']
valores = [0]+[100]+datos+[100-sum(datos)]

In [35]:
a=49
dic_treemap =  datos_para_teemap(df1,2020,3)
fig = go.Figure(go.Treemap(
    labels = dic_treemap['nombres'],
    parents = dic_treemap['padres'],
    branchvalues = 'total',
    values = dic_treemap['datos'],
    sort = True,
     marker=dict(
            colors=dic_treemap['datos']
            )
    
))
fig.update_traces(selector=dict(type='treemap'))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.show()


In [71]:
#a = (['Total exportaciones:']*(len(df_complejos.complejo_exportador)-1)).insert(0, '')

padres = (['Total exportaciones:']*(len(df_complejos.complejo_exportador)-1))
padres.insert(0, '')
print(padres)

['', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:', 'Total exportaciones:']


In [187]:
a = 10
print(df.complejo_exportador[:a].to_list(),lista_padres(df)[:a],df.partim[:a].to_list())

['Total exportaciones:', '---Sector oleaginoso', '------Complejo soja', '---------Harinas y pellets de soja', '---------Aceite de soja', '---------Porotos de soja', '---------Biodiésel', '---------Otras exportaciones de soja', '------Complejo girasol', '---------Aceite de girasol'] ['', 'Total exportaciones:', '------Complejo soja', '------Complejo soja', '------Complejo soja', '------Complejo soja', '------Complejo soja', '------Complejo soja', '------Complejo girasol', '------Complejo girasol'] [100.0, 37.0, 33.6, 18.0, 10.4, 2.8, 1.8, 0.7, 1.9, 1.4]


In [186]:

fig = go.Figure(go.Treemap(
    labels = df.complejo_exportador[:a].to_list(),
    parents = lista_padres(df)[:a],
    branchvalues = 'total',
    values = df.partim[:a].to_list(),
    sort = True,
     marker=dict(
            colors=df.partim[:a].to_list()
            )
    
))
fig.update_traces(selector=dict(type='treemap'))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.show()

In [92]:
fig.write_image("cuadraditos-expo.pdf")
fig.write_html("cuadraditos-expo.html")